# Demo de Extracción desde Socrata

En este notebook se muestra paso a paso cómo utilizar la clase `SocrataDatasetLoader` para extraer datos de Socrata, aplicar validaciones de gobernanza y calidad, y registrar la metadata asociada al proceso. 

Cada celda está comentada para que puedas entender el propósito y el funcionamiento de cada paso.

## Librerias y configuraciones

In [1]:
# Importamos las librerías y módulos necesarios.
from sodapy import Socrata
import sys
import os
import yaml

import warnings
warnings.filterwarnings("ignore", message="Could not infer format")

from dotenv import load_dotenv
load_dotenv()

True

### Configuración del entorno de ejecución

In [2]:
from config.notebook_location import find_project_root

# Definir el nombre del directorio del proyecto
project_name = "personal-library"

# Encontrar la raíz del proyecto
project_root = find_project_root(project_name)

# Agregar la raíz del proyecto al principio de sys.path para facilitar las importaciones
sys.path.insert(0, project_root)
print("Project root added to sys.path:", project_root)

Project root added to sys.path: d:\Espacios de trabajo\personal-library


### Librerías personales

In [3]:
import logging
from ingestion.base.metadata_logger import MetadataLogger

# --------------------------------------------------------------
# Configuración Global del Logging en el Notebook
# --------------------------------------------------------------
# DEBUG: mostrará todos los mensajes
# INFO: mostrará todos los mensajes excepto DEBUG
# WARNING: mostrará todos los mensajes excepto DEBUG y INFO
# ERROR: mostrará todos los mensajes excepto DEBUG, INFO y WARNING
# CRITICAL: mostrará todos los mensajes excepto DEBUG, INFO, WARNING y ERROR
logging.basicConfig(
    level=logging.WARNING,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger()

# Si el logger no tiene handlers asignados, se agrega un StreamHandler para imprimir en consola.
if not logger.handlers:
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.DEBUG)  # Define el nivel mínimo para este handler (DEBUG en este caso)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

# --------------------------------------------------------------
# Inicialización de MetadataLogger
# --------------------------------------------------------------
# Se crea una instancia de MetadataLogger especificando la ruta del reporte y el formato deseado.
ml = MetadataLogger(report_path="reports/s2_contracts.json", file_format="json")

2025-04-07 09:33:33,223 - ingestion.base.metadata_logger - DEBUG - MetadataLogger inicializado con report_path='reports/s2_contracts.json' y file_format='json'


# Ingesta de datos

## Extracción de los datos

In [4]:
from ingestion.sources.from_socrata import SocrataDatasetLoader

# Inicializamos el cliente de Socrata.
client = Socrata("www.datos.gov.co", os.environ["SOCRATA_API_KEY"], timeout=60)

# Creamos la instancia del loader para Socrata.
loader = SocrataDatasetLoader(client)

# Definimos los parámetros para la extracción.
dataset_code = "jbjy-vk9h"  # Código del dataset en Socrata
filters = {"fecha_de_firma": (">=", "2023-01-01")}
limit = 100

ml.log({
    "step": "data_extraction",
    "source": "Socrata",
    "dataset_code": dataset_code,
    "filters": filters,
    "limit": limit,
    "status": "started"
})

# Realizamos la carga de datos.
df = loader.load_data(dataset_code=dataset_code, filters=filters, limit=limit)

ml.log({
    "step": "data_extraction",
    "record_count": len(df),
    "status": "completed"
})

2025-04-06 19:57:14,279 - ingestion.base.metadata_logger - DEBUG - Logged metadata: {'step': 'data_extraction', 'source': 'Socrata', 'dataset_code': 'jbjy-vk9h', 'filters': {'fecha_de_firma': ('>=', '2023-01-01')}, 'limit': 100, 'status': 'started', 'uuid': 'fb080d32-5e52-4a25-b1e2-57aad0b54725', 'timestamp': '2025-04-06T23:57:14.279095+00:00'}


2025-04-06 19:57:17,410 - ingestion.base.metadata_logger - DEBUG - Logged metadata: {'step': 'data_extraction', 'record_count': 100, 'status': 'completed', 'uuid': '9bacce1d-d839-4bb9-b7d2-cb58837d89b3', 'timestamp': '2025-04-06T23:57:17.410841+00:00'}


## Muestreo

In [5]:
from ingestion.base.sampler import Sampler

# Creamos un sampler
sampler = Sampler(data=df)

# Obtenemos el tamaño del sample
sampler_size = sampler.get_sample_size()

df_sample = sampler.perform_sampling(seed=42)

## Métricas

### Estadísticos y medidas básicas de calidad

In [6]:
from governance.quality_management.data_quality_checks.advance_quality_report import QualityReportEngine

# Se crea el motor de calidad
engine = QualityReportEngine(df_sample)

# Se genera el reporte
report = engine.generate_report()

2025-04-06 19:57:19,821 - governance.quality_management.data_quality_checks.advance_quality_report - WARNING - safe_nunique: Se encontró un elemento no hashable, convirtiendo a string.


In [7]:
from governance.quality_management.data_remediation.iterative_remediation_engine import IterativeRemediationEngine

# Listas opcionales de campos a incluir/excluir
include_fields = None  # Ejemplo: ["numeros", "texto"]
exclude_fields = None  # Ejemplo: ["fechas"]

config = {
    "max_epochs": 5,
    "improvement_threshold": 0.5,
    "knowledge_file": "remediation_knowledge.json",
    "include_fields": None,
    "exclude_fields": None
}

# Crear el motor iterativo (la configuración se carga desde remediation_config.json si existe)
iterative_engine = IterativeRemediationEngine(df_sample, config=config)
final_df, logs, knowledge = iterative_engine.run()

2025-04-06 19:57:22,358 - governance.quality_management.data_quality_checks.advance_quality_report - WARNING - safe_nunique: Se encontró un elemento no hashable, convirtiendo a string.
2025-04-06 19:58:34,267 - governance.quality_management.data_remediation.imputation.performance - WARNING - No hay valores originales válidos para evaluar.
2025-04-06 19:58:34,308 - governance.quality_management.data_remediation.imputation.performance - WARNING - No hay valores originales válidos para evaluar.
d:\Espacios de trabajo\personal-library\governance\quality_management\data_remediation\bias\rules\datetime.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<TimedeltaArray>
[NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT,
 NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT,
 NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT, NaT,
 NaT, NaT, NaT,

In [9]:
final_df

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,...,nombre_ordenador_del_gasto,tipo_de_documento_ordenador_del_gasto,n_mero_de_documento_ordenador_del_gasto,nombre_supervisor,tipo_de_documento_supervisor,n_mero_de_documento_supervisor,nombre_ordenador_de_pago,tipo_de_documento_ordenador_de_pago,n_mero_de_documento_ordenador_de_pago,fecha_de_notificaci_n_de_prorrogaci_n
83,hospital departamental universitario santa sof...,890801099,caldas,manizales,colombia caldas manizales,territorial,salud proteccion social,ejecutivo,descentralizada,co1.bdos.3659336,...,carlos alberto piedrahita gutierrez,cedula ciudadania,9845752 .,jose ignacio londoño jimenez,no definido,no definido,no definido,no definido,no definido,1970-01-01
53,hospital san carlos saldaña,890701300,tolima,saldaña,colombia tolima saldaña,territorial,salud proteccion social,ejecutivo,descentralizada,co1.bdos.3685776,...,agustin antonio jacobs vizcaino,cedula ciudadania,19615145,maria alejandra sabogal angel,cedula ciudadania,1108937449,no definido,no definido,no definido,1970-01-01
70,hospital local municipio patios,807004393,norte santander,patios,colombia norte santander patios,nacional,cultura,ejecutivo,centralizada,co1.bdos.3688647,...,liliana elena rodriguez pelaez,cedula ciudadania,60332458,juan carlos fonseca duarte,cedula ciudadania,88193856,no definido,no definido,no definido,2023-04-28
45,hospital susana lopez valencia,891501676,cauca,popayan,colombia cauca popayan,territorial,salud proteccion social,corporacion autonoma,descentralizada,co1.bdos.3689555,...,edgar eduardo villa,cedula ciudadania,76310238,jorge javier ñañez hoyos,cedula ciudadania,76320338,no definido,no definido,no definido,2023-07-01
44,hospital san carlos saldaña,890701300,tolima,saldaña,colombia tolima saldaña,territorial,salud proteccion social,ejecutivo,descentralizada,co1.bdos.3685771,...,agustin antonio jacobs vizcaino,cedula ciudadania,19615145,viviana andrea homez rodriguez,cedula ciudadania,65589349,no definido,no definido,no definido,1970-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,hospital san rafael espinal tolima e.s.e,890701033,tolima,espinal,colombia tolima espinal,territorial,salud proteccion social,ejecutivo,descentralizada,co1.bdos.3690707,...,carmen patricia henao max,cedula ciudadania,30328159,alejandro doncel barrios,cedula ciudadania,79372962,no definido,no definido,no definido,1970-01-01
75,empresa social region salud soacha .,8000068503,cundinamarca,soacha,colombia cundinamarca soacha,territorial,salud proteccion social,corporacion autonoma,centralizada,co1.bdos.3686319,...,alexandra gonzalez moreno,cedula ciudadania,8000068503,no definido,no definido,no definido,no definido,no definido,no definido,2023-01-01
32,hospital departamental san juan dios puerto ca...,842000004,vichada,puerto carreño,colombia vichada puerto carreño,territorial,salud proteccion social,corporacion autonoma,centralizada,co1.bdos.3683873,...,zamir enrique zamora florez,cedula ciudadania,73571579,fanny adelaida muñoz giraldo,cedula ciudadania,30215829,no definido,no definido,no definido,1970-01-01
94,e.s.e hospital arsenio repizo vanegas,891180113,huila,san agustin,colombia huila san agustin,territorial,salud proteccion social,corporacion autonoma,descentralizada,co1.bdos.3689015,...,eseharv2,cedula ciudadania,55182626,alejandra barragan santofimio,cedula ciudadania,55179938,no definido,no definido,no definido,2023-06-15


**3. Carga de la Política de Gobernanza**

La política define las reglas de calidad y gobernabilidad que se aplicarán al dataset.

Se carga desde un archivo YAML. Asegúrate de que el archivo `s2_contracts.yaml` exista en la ruta indicada.

In [ ]:
policy = get_or_create_policy(df, "s2_contracts.yaml")

In [ ]:
# Imprimimos la política cargada para verificar su contenido.
print("Política de Gobernanza Cargada:")
print(yaml.dump(policy, sort_keys=False, allow_unicode=True))

Política de Gobernanza Cargada:
dataset_metadata:
  columns: 85
  rows_sampled: 100
  generated_by: default_policy_generator
  generated_on: '2025-03-29T22:30:23.876640Z'
  data_source: inferred
  schema_version: '1.0'
  language: undetected
compliance:
  compliance_frameworks:
  - GDPR
  risk_level: high
enforcement_requirements:
  mandatory_rules:
  - no_nulls
  - valid_date_format
  required_fields: []
  privacy_enforcement: medium
  security_baseline: masked
  allowed_transparency:
  - internal
  - public
  risk_acceptance: medium
  framework_enforcement:
  - GDPR
fields:
- field_name: nombre_entidad
  type: string
  required: true
  rules: []
  privacy_level: high
  security: encrypted
  transparency: internal
  integrity:
    unique: false
    no_nulls: true
    consistent_format: true
    contains_outliers: false
  compliance_tags:
  - GDPR
  data_subject: true
  access_restriction: restricted
  retention_policy: default
  critical_field: false
  justification: Field 'nombre_ent

In [ ]:
from governance.automation_and_monitoring.real_time_monitoring.data_quality_monitor import DataQualityMonitor
from governance.automation_and_monitoring.real_time_monitoring.metric_registry import MetricRegistry

In [ ]:
registry = MetricRegistry()

In [ ]:
monitor = DataQualityMonitor(df, "s2_contracts.yaml", registry=registry)

2025-04-02 10:46:09,476 [INFO] Política cargada mediante get_or_create_policy.


In [ ]:
# Evaluar la fase de Ingesta y mostrar sus métricas.
ingestion_metrics = monitor.evaluate_phase("ingestion")
print("Métricas de Ingesta:")
print(ingestion_metrics['field_metrics']['global'])

2025-04-02 10:46:09,652 [INFO] Fase de Ingesta completada.


Métricas de Ingesta:
{'average_quality_score': np.float64(69.75396935768111), 'total_fields': 85}


In [ ]:
monitor.generate_report()

2025-04-02 10:46:09,831 [INFO] Fase de Ingesta completada.
2025-04-02 10:46:09,832 [INFO] Fase de Desagregación completada.
2025-04-02 10:46:09,949 [INFO] Fase de Limpieza completada.
2025-04-02 10:46:09,951 [INFO] Fase de Indexación completada.
2025-04-02 10:46:10,113 [INFO] Monitoreo Continuo completado.


{'ingestion': {'gdpr_compliance': 'Compliant',
  'field_metrics': {'nombre_entidad': {'null_percentage': np.float64(0.0),
    'type_match': True,
    'duplicate_percentage': 69.0,
    'uniqueness_rate': 0.31,
    'contains_outliers': False,
    'mean': None,
    'median': None,
    'std': None,
    'skewness': None,
    'percentiles': {},
    'outlier_percentage': None,
    'temporal_anomaly': None,
    'cardinality_ratio': 0.31,
    'security_compliant': np.False_,
    'relational_compliance': 'n/a',
    'field_quality_score': np.float64(86.2)},
   'nit_entidad': {'null_percentage': np.float64(0.0),
    'type_match': False,
    'duplicate_percentage': 69.0,
    'uniqueness_rate': 0.31,
    'contains_outliers': False,
    'mean': np.float64(1101537680.46),
    'median': np.float64(890751787.5),
    'std': np.float64(1329078600.2603405),
    'percentiles': {'25': np.float64(842000004.0),
     '50': np.float64(890751787.5),
     '75': np.float64(890980093.0)},
    'skewness': np.float64(

**4. Validación del Dataset con el Motor de Gobernanza**

Se utiliza la clase `GovernanceEngine` para aplicar las validaciones definidas en la política al DataFrame.

Se generan reportes que incluyen errores y advertencias detectadas en la ingesta.

In [ ]:
# Crea una instancia del engine usando el nombre de política deseado
engine = GovernanceEngine(df, "s2_contracts.yaml")
    
# Ejecuta las validaciones y muestra las advertencias
warnings = engine.run_policy_checks()
if warnings:
    print("Warnings generated by GovernanceEngine:")
    for w in warnings:
        print(" -", w)
else:
    print("No warnings. The dataset complies with the policy.")

Warnings generated by GovernanceEngine:
 - {'field': 'nombre_entidad', 'issue': "Missing mandatory rule 'no_nulls'.", 'severity': 'warning', 'execution_id': '043d123a-e010-475c-9988-3e1599b8a975', 'timestamp': '2025-04-02T11:30:23.873363'}
 - {'field': 'nombre_entidad', 'issue': "Missing mandatory rule 'valid_date_format'.", 'severity': 'warning', 'execution_id': '043d123a-e010-475c-9988-3e1599b8a975', 'timestamp': '2025-04-02T11:30:23.873363'}
 - {'field': 'nit_entidad', 'issue': "Expected type 'integer' but got 'string'.", 'severity': 'warning', 'execution_id': '043d123a-e010-475c-9988-3e1599b8a975', 'timestamp': '2025-04-02T11:30:23.873363'}
 - {'field': 'nit_entidad', 'issue': "Missing mandatory rule 'no_nulls'.", 'severity': 'warning', 'execution_id': '043d123a-e010-475c-9988-3e1599b8a975', 'timestamp': '2025-04-02T11:30:23.873363'}
 - {'field': 'nit_entidad', 'issue': "Missing mandatory rule 'valid_date_format'.", 'severity': 'warning', 'execution_id': '043d123a-e010-475c-9988-3e

In [ ]:
meta_logger.log({
    "step": "policy_application",
    "policy_name": "s2_contracts.yaml",
    "status": "applied"
})

DEBUG:ingestion.base.metadata_logger:Logged metadata: {'step': 'policy_application', 'policy_name': 's2_contracts.yaml', 'status': 'applied', 'uuid': 'c3ab5862-1e8f-4dc9-8ea5-7ea3b40eee18', 'timestamp': '2025-04-02T11:30:25.758528+00:00'}


In [ ]:
metrics_engine = MetricsEngine(df, "s2_contracts.yaml")
metrics_dict = metrics_engine.generate_quality_metrics()

In [ ]:
meta_logger.log({
    "step": "governance_metrics",
    "status": "executed"
})

DEBUG:ingestion.base.metadata_logger:Logged metadata: {'step': 'governance_metrics', 'status': 'executed', 'uuid': '2c326c9d-d661-45e5-901a-f1ead3e50bc9', 'timestamp': '2025-04-02T11:30:28.483311+00:00'}


In [ ]:
metrics_dict

{'nombre_entidad': {'null_percentage': np.float64(0.0),
  'type_match': True,
  'duplicate_percentage': 69.0,
  'uniqueness_rate': 0.31,
  'contains_outliers': False,
  'mean': None,
  'median': None,
  'std': None,
  'skewness': None,
  'percentiles': {},
  'outlier_percentage': None,
  'temporal_anomaly': None,
  'cardinality_ratio': 0.31,
  'security_compliant': np.False_,
  'relational_compliance': 'n/a',
  'field_quality_score': np.float64(86.2)},
 'nit_entidad': {'null_percentage': np.float64(0.0),
  'type_match': False,
  'duplicate_percentage': 69.0,
  'uniqueness_rate': 0.31,
  'contains_outliers': False,
  'mean': np.float64(1101537680.46),
  'median': np.float64(890751787.5),
  'std': np.float64(1329078600.2603405),
  'percentiles': {'25': np.float64(842000004.0),
   '50': np.float64(890751787.5),
   '75': np.float64(890980093.0)},
  'skewness': np.float64(5.615650057510579),
  'outlier_percentage': np.float64(3.0),
  'temporal_anomaly': None,
  'cardinality_ratio': None,
  

In [ ]:
remediation_engine = RemediationEngine(df, "s2_contracts.yaml")
df_procesed = remediation_engine.run_remediation()

In [ ]:
df_procesed.head()

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,...,nombre_ordenador_del_gasto,tipo_de_documento_ordenador_del_gasto,n_mero_de_documento_ordenador_del_gasto,nombre_supervisor,tipo_de_documento_supervisor,n_mero_de_documento_supervisor,nombre_ordenador_de_pago,tipo_de_documento_ordenador_de_pago,n_mero_de_documento_ordenador_de_pago,fecha_de_notificaci_n_de_prorrogaci_n
0,SE9TUElUQUwgU0FOIEpVQU4gREUgRElPUyBERSBIT05EQS...,ODkwNzAwNjY2,Tolima,Honda,"Colombia, Tolima , Honda",Territorial,Salud y Protección Social,Corporación Autónoma,RGVzY2VudHJhbGl6YWRh,Q08xLkJET1MuMzY4MTIyMw==,...,MANUEL ALFONSO GONZaLEZ CANTOR,Cédula de Ciudadanía,NzkzOTMxNzI=,MARTHA LUCIA OSORIO RAMIREZ,Cédula de Ciudadanía,MzgyODEzMTg=,No definido,No definido,No definido,NaT
1,SE9TUElUQUwgREUgQ0FTVElMTEEgTEEgTlVFVkEgRU1QUk...,OTAwMDA0MDU5,Meta,Castilla La Nueva,"Colombia, Meta , Castilla La Nueva",Territorial,Salud y Protección Social,Corporación Autónoma,RGVzY2VudHJhbGl6YWRh,Q08xLkJET1MuMzY5MDUyOA==,...,ROSA MARIA JIMENEZ BAQUERO,Cédula de Ciudadanía,NDAzNzA4OTM=,LUIS ENRIQUE BARON TELLO,Cédula de Ciudadanía,MTEyMTgyNzM1Mw==,No definido,No definido,No definido,NaT
2,SE9TUElUQUwgU0FOIEpVQU4gREUgRElPUyBERSBIT05EQS...,ODkwNzAwNjY2,Tolima,Honda,"Colombia, Tolima , Honda",Territorial,Salud y Protección Social,Corporación Autónoma,RGVzY2VudHJhbGl6YWRh,Q08xLkJET1MuMzY3NzYyNA==,...,MANUEL ALFONSO GONZaLEZ CANTOR,Cédula de Ciudadanía,NzkzOTMxNzI=,ANGELA MARLEN SALGUERO RAMIREZ,Cédula de Ciudadanía,MTExMDUyNjM2NQ==,No definido,No definido,No definido,NaT
3,RU1QUkVTQSBTT0NJQUwgREVMIEVTVEFETyBIT1NQSVRBTC...,OTY0NDUwMjI2,Antioquia,Amalfi,"Colombia, Antioquia , Amalfi",Territorial,Salud y Protección Social,Ejecutivo,RGVzY2VudHJhbGl6YWRh,Q08xLkJET1MuMzY4ODM2NQ==,...,LICINIA DEL CARMEN RAVE BERMUDEZ,Cédula de Ciudadanía,NDI4NzY1MTY=,ANGEL ERNESTO FRANCO HENAO,Cédula de Ciudadanía,NzAyNTIxOTY=,No definido,No definido,No definido,NaT
4,TVVOSUNJUElPIERFIElUQUdVSQ==,ODkwOTgwMDkz,Antioquia,Itagui,"Colombia, Antioquia , Itagui",Territorial,"Vivienda, Ciudad y Territorio",Ejecutivo,Q2VudHJhbGl6YWRh,Q08xLkJET1MuMzY3MTQ0MQ==,...,No definido,No definido,Tm8gZGVmaW5pZG8=,No definido,No definido,Tm8gZGVmaW5pZG8=,No definido,No definido,No definido,NaT


In [ ]:
meta_logger.log({
    "step": "remediation",
    "status": "executed"
})

In [ ]:
metrics_engine_procesed = MetricsEngine(df_procesed, "s2_contracts.yaml")
metrics_dict_df_processed = metrics_engine_procesed.generate_quality_metrics()

d:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Espacios de trabajo\pe

In [ ]:
metrics_dict_df_processed

{'nombre_entidad': {'null_percentage': np.float64(0.0),
  'type_match': True,
  'duplicate_percentage': 69.0,
  'uniqueness_rate': 0.31,
  'contains_outliers': False,
  'mean': None,
  'median': None,
  'std': None,
  'skewness': None,
  'percentiles': {},
  'outlier_percentage': None,
  'temporal_anomaly': None,
  'cardinality_ratio': 0.31,
  'security_compliant': np.True_,
  'relational_compliance': 'n/a',
  'field_quality_score': np.float64(86.2)},
 'nit_entidad': {'null_percentage': np.float64(0.0),
  'type_match': False,
  'duplicate_percentage': 70.0,
  'uniqueness_rate': 0.3,
  'contains_outliers': False,
  'mean': nan,
  'median': np.float64(nan),
  'std': np.float64(nan),
  'percentiles': {'25': np.float64(nan),
   '50': np.float64(nan),
   '75': np.float64(nan)},
  'skewness': nan,
  'outlier_percentage': np.float64(0.0),
  'temporal_anomaly': None,
  'cardinality_ratio': None,
  'security_compliant': True,
  'relational_compliance': 'n/a',
  'field_quality_score': np.float64

In [ ]:
meta_logger.log({
    "step": "governance_metrics",
    "status": "executed"
})

In [ ]:
engine = IntelligentImprovementEngine(metrics_dict_df_processed, metrics_engine.policy)
recommendations = engine.generate_improvement_recommendations()

print("Global Recommendations:")
for rec in recommendations["global"]:
    print(" -", rec)
for field, recs in recommendations["fields"].items():
    print(f"Field '{field}':")
    for r in recs:
        print("   -", r)

Global Recommendations:
 - La calidad global es baja; se recomienda una revisión integral de la ingesta.
Field 'nombre_entidad':
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 13.8 puntos.
Field 'nit_entidad':
   - Optimizar 'type_match' podría incrementar el score en hasta 20.0 puntos.
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 14.0 puntos.
Field 'departamento':
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 17.4 puntos.
Field 'ciudad':
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 13.8 puntos.
Field 'localizaci_n':
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 13.8 puntos.
Field 'orden':
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 19.6 puntos.
Field 'sector':
   - Optimizar 'duplicate_percentage' podría incrementar el score en hasta 19.4 puntos.
Field 'rama':
   - Optimizar 'duplicate_percentage' podría increm

**5. Registro de Metadata y Auditoría**
 
Utilizamos la clase `MetadataLogger` para registrar la metadata del proceso de ingesta, incluyendo:

- Información del loader (número de filas, estado, filtros aplicados, etc.)

- Reporte de gobernanza (errores y advertencias)

La metadata se guarda en un archivo Parquet para su posterior auditoría y seguimiento.

In [ ]:
meta_logger.save()

2025-03-28 08:00:49,794 - ingestion.base.metadata_logger - DEBUG - Logged metadata: {'source': 'socrata', 'status': 'success', 'error': None, 'row_count': 100, 'timestamp': '2025-03-28T12:00:49.794419+00:00', 'filters_applied': {'fecha_de_firma': ('>=', '2023-01-01')}, 'dataset_code': 'jbjy-vk9h', 'errors': [], 'warnings': ["El campo 'fecha_de_firma' tiene tipo 'object', se esperaba 'datetime64'."], 'uuid': '5c7be947-9e9c-4264-abf0-2e591051d2e8'}
2025-03-28 08:00:50,830 - ingestion.base.metadata_logger - INFO - Metadata log saved to reports/demo_socrata.parquet


Metadata registrada y audit log guardado.
